In [78]:
#import libraries
#Need to pip install selenium
from splinter import Browser
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import date

In [79]:
# Set up splinter
browser = Browser('chrome')

# base url
base_url = "https://www.facebook.com/marketplace/sanfrancisco/search?"

# search parameters
# days_listed = 7
deliveryMethod = "local_pick_up"
query = "honda%20insight"
availability = "in%20stock"

#full url
url = f"{base_url}availability={availability}&deliveryMethod={deliveryMethod}&query={query}"

In [80]:
#visit site
browser.visit(url)

In [81]:
if browser.is_element_present_by_css('div[aria-label="Close"]', wait_time=10):
    # Click on the element once it's found
    browser.find_by_css('div[aria-label="Close"]').first.click()

In [82]:
# scroll down to load more results

scroll_count = 4

scroll_delay = 2

for _ in range(scroll_count):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(scroll_delay)

In [83]:
# Parse HTML
html = browser.html

#create BS object from HTML
market_soup = soup(html, "html.parser")

In [84]:
# End browsing session
browser.quit()

In [85]:
# Extract all the info, put into lists
titles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
titles_list = [title.text.strip() for title in titles_div]
titles_list.pop(0)
titles_list.pop(0)

prices_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs xlh3980 xvmahel x1n0sxbx x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1lkfr7t x1lbecb7 x1s688f xzsf02u")
prices_list = [price.text.strip() for price in prices_div]

miles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6 xlyipyv xuxw1ft")
miles_list = [mile.text.strip() for mile in miles_div]

image_elems = market_soup.find_all('img', class_= "xt7dq6l xl1xv1r x6ikm8r x10wlt62 xh8yej3")
image_list = [image.get('src') for image in image_elems]

urls_elems = market_soup.find_all('a', class_= "x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv")
urls_list = [url.get('href') for url in urls_elems]




In [86]:
titles_list

['2010 Honda insight EX Hatchback 4D',
 '2020 Honda insight Touring Sedan 4D',
 '2002 Honda insight Hybrid Hatchback 2D',
 '2022 Honda insight EX Sedan 4D',
 '2010 Honda insight EX Hatchback 4D',
 '2010 Honda insight LX Hatchback 4D',
 '2010 Honda insight Hybrid',
 '2019 Honda insight Touring Sedan 4D',
 '2010 Honda insight EX Hatchback 4D',
 '2020 Honda insight',
 '2019 Honda insight Touring Sedan 4D',
 '2000 Honda insight - stick shift',
 '2010 Honda insight LX Hatchback 4D',
 '2019 Honda insight LX Sedan 4D',
 'Honda crz',
 '2005 Honda insight',
 '2012 Honda insight Hatchback 4D',
 '2002 Honda insight Hybrid Hatchback 2D',
 '2010 Honda insight EX Hatchback 4D',
 '2003 Honda insight Hatchback 2D',
 '2000 Honda insight',
 '2019 Honda insight',
 '2020 Honda insight EX Sedan 4D',
 '2012 Honda civic',
 '2012 Honda insight Hatchback 4D',
 '2010 Honda insight',
 '2010 Honda insight LX Hatchback 4D',
 '2011 Honda insight Hatchback 4D',
 '2011 2011 Honda Insight hybrid insight',
 '2012 Honda

In [87]:
image_list

['https://scontent-sjc3-1.xx.fbcdn.net/v/t39.30808-6/433141191_7307447259303959_6561256881350902246_n.jpg?stp=c0.29.261.261a_dst-jpg_p261x260&_nc_cat=107&ccb=1-7&_nc_sid=5f2048&_nc_ohc=NpIsoE2sQNgAX8IOzVN&_nc_ht=scontent-sjc3-1.xx&oh=00_AfB6JNuI9FDNi6DuinbkURGIRk7IgAjrsJzhANZmEZorhg&oe=660B293B',
 'https://scontent-sjc3-1.xx.fbcdn.net/v/t39.30808-6/433085950_122135765186054836_9042306065833971813_n.jpg?stp=c43.0.260.260a_dst-jpg_p261x260&_nc_cat=106&ccb=1-7&_nc_sid=5f2048&_nc_ohc=Z7BCtvZMOYwAX-_9KVD&_nc_ht=scontent-sjc3-1.xx&oh=00_AfAJLC6sRQ8WRYo54ip26gTjbsE0VowqZXWAwI-3Xi3kMQ&oe=660B6BFC',
 'https://scontent-sjc3-1.xx.fbcdn.net/v/t39.30808-6/433140773_122115755444240073_5556977948482205665_n.jpg?stp=c0.86.828.828a_dst-jpg_s261x260&_nc_cat=100&ccb=1-7&_nc_sid=5f2048&_nc_ohc=vGBiUQa07jgAX955Bzt&_nc_ht=scontent-sjc3-1.xx&oh=00_AfB22luc9B201auZlIbzaMP-WCIHwhwu3Nt165B-h5Sd2w&oe=660D13CF',
 'https://scontent-sjc3-1.xx.fbcdn.net/v/t45.5328-4/431914426_428965812871237_2556074385288478216_n.jp

In [88]:
miles_list

['Log In',
 'Log In',
 'Notify Me',
 'Create new listing',
 'Sort by',
 'Delivery method: Local pickup',
 'Condition',
 'Date listed',
 'Availability',
 'San Francisco · 40 mi',
 'Filters',
 'Notify Me',
 'San Mateo, CA',
 '164K miles',
 'Benicia, CA',
 '62K miles',
 'San Francisco, CA',
 '165K miles',
 'San Francisco, CA',
 '24K miles',
 'Fremont, CA',
 '183K miles',
 'San Francisco, CA',
 '184K miles',
 'Hayward, CA',
 '145K miles',
 'San Jose, CA',
 '90K miles',
 'Manteca, CA',
 '154K miles',
 'San Rafael, CA',
 '37K miles',
 'Carmichael, CA',
 '49K miles',
 'San Jose, CA',
 '205K miles',
 'Sacramento, CA',
 '179K miles',
 'Antioch, CA',
 '46K miles',
 'San Francisco, CA',
 'Redwood City, CA',
 '209K miles',
 'Carmel-by-the-Sea, CA',
 '173K miles',
 'San Jose, CA',
 '271K miles',
 'Sacramento, CA',
 '52K miles',
 'Citrus Heights, CA',
 '22K miles',
 'San Jose, CA',
 '205K miles',
 'Roseville, CA',
 '28K miles',
 'Sacramento, CA',
 '48K miles',
 'Petaluma, CA',
 '134K miles',
 'San J

In [89]:
urls_list

['/marketplace/item/392001890437836/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/1543892023069552/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/1492382191345101/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/438906411912035/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/313640594752406/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/425378283505584/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/1324872828208980/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/973014317516704/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/362982763334192/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/12011947775241

In [90]:
# regex to filter
pattern = re.compile(r'[a-zA-Z ]+[,]\s[A-Z][A-Z]')
miles_pattern = re.compile(r'([0-9.]+)[K]? miles')

# initialize empty list
miles_list2 = []

#iterate through the original mileage entries
for item in miles_list:
    # miles_list2.append(item)

    if pattern.match(item) and len(miles_list2) >= 1 and pattern.match(miles_list2[-1]):
        miles_list2.append('0K miles')
        miles_list2.append(item)

    elif len(item) == 0 and pattern.match(miles_list2[-1]) and len(miles_list2) >=1:
        miles_list2.append('0K miles')

    elif len(miles_list2) >= 1 and pattern.match(miles_list2[-1]) and miles_pattern.match(item) == None:
        miles_list2.append('0K miles')

    else:
        miles_list2.append(item)



In [91]:
prices_list

['$4,000',
 '$15,900',
 '$3,650',
 '$24,500',
 '$4,500',
 '$6,200',
 '$4,900',
 '$2,500',
 '$6,800',
 '$23,000',
 '$15,500',
 '$4,500',
 '$4,050',
 '$15,950',
 '$4,000',
 '$1,500',
 '$4,799',
 '$2,500',
 '$7,500',
 '$4,000',
 '$4,500',
 '$16,200',
 '$17,300',
 '$6,600',
 '$6,100',
 '$3,240',
 '$3,650',
 '$5,500',
 '$5,250',
 '$4,900',
 '$5,898',
 '$2,498',
 '$4,500',
 '$16,199',
 '$7,200',
 '$5,900',
 '$7,300',
 '$1',
 'MX$1,234',
 '$4,500',
 '$555',
 '$8,799',
 '$555',
 '$100',
 '$19,500',
 '$3,500',
 '$9,800',
 '$8,000',
 '$555',
 '$18,800',
 '$13,950',
 '$3,900',
 '$6,950',
 '$4,000',
 '$1,000',
 '$9,900',
 '$20,000',
 '$99',
 '$450',
 '$5,000',
 '$18,000',
 '$7,500',
 '$8,000',
 '$5,000',
 '$19,995',
 '$250',
 '$4,500',
 '$50',
 '$7,900',
 '$21,499',
 '$11,800',
 '$8,500',
 '$8,500',
 '$18,900',
 '$500',
 '$550',
 '$19,777',
 '$4,500',
 '$6,000',
 '$13,000',
 '$1,500',
 '$22,999',
 '$7,900',
 '$4,500',
 '$23,500',
 '$5,500',
 '$2,200',
 '$4,150',
 '$14,500',
 '$1,500',
 '$3,800',
 

In [92]:
# Clean Location and Miles Data
miles_pattern_miles = r'([0-9.]+)[K]? miles'
miles_pattern_km = r'(\d+)K km'
location_pattern = r'[a-zA-Z ]+[,]\s[A-Z][A-Z]'

miles_clean = []
locations_clean =[]

for item in miles_list2:

    match_miles_km = re.search(miles_pattern_km, item)

    match_mileage_miles = re.search(miles_pattern_miles, item)

    match_location = re.search(location_pattern, item)

    if match_miles_km or match_mileage_miles or match_location:
        if match_miles_km:
            miles_clean.append(int(match_miles_km.group(1)) *1000)

        if match_mileage_miles:
            miles_clean.append(int(float(match_mileage_miles.group(1))) *1000)
            # print(match_mileage_miles)
            # print(int(float(match_mileage_miles.group(1))) *1000)

        if match_location:
            locations_clean.append(item)
            print(match_location)
            print(item)

<re.Match object; span=(0, 13), match='San Mateo, CA'>
San Mateo, CA
<re.Match object; span=(0, 11), match='Benicia, CA'>
Benicia, CA
<re.Match object; span=(0, 17), match='San Francisco, CA'>
San Francisco, CA
<re.Match object; span=(0, 17), match='San Francisco, CA'>
San Francisco, CA
<re.Match object; span=(0, 11), match='Fremont, CA'>
Fremont, CA
<re.Match object; span=(0, 17), match='San Francisco, CA'>
San Francisco, CA
<re.Match object; span=(0, 11), match='Hayward, CA'>
Hayward, CA
<re.Match object; span=(0, 12), match='San Jose, CA'>
San Jose, CA
<re.Match object; span=(0, 11), match='Manteca, CA'>
Manteca, CA
<re.Match object; span=(0, 14), match='San Rafael, CA'>
San Rafael, CA
<re.Match object; span=(0, 14), match='Carmichael, CA'>
Carmichael, CA
<re.Match object; span=(0, 12), match='San Jose, CA'>
San Jose, CA
<re.Match object; span=(0, 14), match='Sacramento, CA'>
Sacramento, CA
<re.Match object; span=(0, 11), match='Antioch, CA'>
Antioch, CA
<re.Match object; span=(0, 1

In [94]:
# Make Prices into Integer
prices_clean = []

for price in prices_list:
    if price == 'Free':
        prices_clean.append(0)
    else:
        prices_clean.append(int(re.sub(r'[₹,M,X,$,]','', price)))


In [95]:
# Make URLS full url
urls_clean = []

for url in urls_list:
    urls_clean.append('https://www.facebook.com' + url)


In [96]:
miles_list2

['Log In',
 'Log In',
 'Notify Me',
 'Create new listing',
 'Sort by',
 'Delivery method: Local pickup',
 'Condition',
 'Date listed',
 'Availability',
 'San Francisco · 40 mi',
 'Filters',
 'Notify Me',
 'San Mateo, CA',
 '164K miles',
 'Benicia, CA',
 '62K miles',
 'San Francisco, CA',
 '165K miles',
 'San Francisco, CA',
 '24K miles',
 'Fremont, CA',
 '183K miles',
 'San Francisco, CA',
 '184K miles',
 'Hayward, CA',
 '145K miles',
 'San Jose, CA',
 '90K miles',
 'Manteca, CA',
 '154K miles',
 'San Rafael, CA',
 '37K miles',
 'Carmichael, CA',
 '49K miles',
 'San Jose, CA',
 '205K miles',
 'Sacramento, CA',
 '179K miles',
 'Antioch, CA',
 '46K miles',
 'San Francisco, CA',
 '0K miles',
 'Redwood City, CA',
 '209K miles',
 'Carmel-by-the-Sea, CA',
 '173K miles',
 'San Jose, CA',
 '271K miles',
 'Sacramento, CA',
 '52K miles',
 'Citrus Heights, CA',
 '22K miles',
 'San Jose, CA',
 '205K miles',
 'Roseville, CA',
 '28K miles',
 'Sacramento, CA',
 '48K miles',
 'Petaluma, CA',
 '134K mi

In [97]:
# add all values to a list of dictionaries
vehicles_list = []

for i, item in enumerate(titles_list):
    cars_dict = {}
    first_gen = False
    insight = False
    parts = False
    years = ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "1st", "first gen"]

    #Checks if any cars are 2000 - 2006
    if any(x in titles_list[i] for x in years):
        first_gen = True

    #Checks if car listing is for parts
    if "part" in titles_list[i].lower():
        parts = True

    #Checks if insight is actually in the title
    if "insight" in titles_list[i].lower():
        insight = True

    #Splits up the City and State from location
    city = locations_clean[i].split(', ')[0]
    state = locations_clean[i].split(', ')[-1]



    # Map out key value pairs
    cars_dict["date"] = date.today()
    cars_dict["title"] = titles_list[i]
    cars_dict["price"] = prices_clean[i]
    cars_dict["city"] = city
    cars_dict["state"] = state
    cars_dict["mileage"] = miles_clean[i]
    cars_dict["url"] = urls_clean[i]
    cars_dict["image"] = image_list[i]
    cars_dict["insight"] = insight
    cars_dict["first_gen"] = first_gen
    cars_dict["parts"] = parts
    cars_dict["site"] = "fb"
    vehicles_list.append(cars_dict)


In [98]:
vehicles_list

[{'date': datetime.date(2024, 3, 29),
  'title': '2010 Honda insight EX Hatchback 4D',
  'price': 4000,
  'city': 'San Mateo',
  'state': 'CA',
  'mileage': 164000,
  'url': 'https://www.facebook.com/marketplace/item/392001890437836/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'image': 'https://scontent-sjc3-1.xx.fbcdn.net/v/t39.30808-6/433141191_7307447259303959_6561256881350902246_n.jpg?stp=c0.29.261.261a_dst-jpg_p261x260&_nc_cat=107&ccb=1-7&_nc_sid=5f2048&_nc_ohc=NpIsoE2sQNgAX8IOzVN&_nc_ht=scontent-sjc3-1.xx&oh=00_AfB6JNuI9FDNi6DuinbkURGIRk7IgAjrsJzhANZmEZorhg&oe=660B293B',
  'insight': True,
  'first_gen': False,
  'parts': False,
  'site': 'fb'},
 {'date': datetime.date(2024, 3, 29),
  'title': '2020 Honda insight Touring Sedan 4D',
  'price': 15900,
  'city': 'Benicia',
  'state': 'CA',
  'mileage': 62000,
  'url': 'https://www.facebook.com/marketplace/item/1543892023069552/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'image

In [99]:
vehicles_df = pd.DataFrame(vehicles_list)

In [100]:
vehicles_df

,date,title,price,city,state,mileage,url,image,insight,first_gen,parts,site
0,2024-03-29,2010 Honda insight EX Hatchback 4D,4000,San Mateo,CA,164000,https://www.facebook.com/marketplace/item/3920...,https://scontent-sjc3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
1,2024-03-29,2020 Honda insight Touring Sedan 4D,15900,Benicia,CA,62000,https://www.facebook.com/marketplace/item/1543...,https://scontent-sjc3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
2,2024-03-29,2002 Honda insight Hybrid Hatchback 2D,3650,San Francisco,CA,165000,https://www.facebook.com/marketplace/item/1492...,https://scontent-sjc3-1.xx.fbcdn.net/v/t39.308...,True,True,False,fb
3,2024-03-29,2022 Honda insight EX Sedan 4D,24500,San Francisco,CA,24000,https://www.facebook.com/marketplace/item/4389...,https://scontent-sjc3-1.xx.fbcdn.net/v/t45.532...,True,False,False,fb
4,2024-03-29,2010 Honda insight EX Hatchback 4D,4500,Fremont,CA,183000,https://www.facebook.com/marketplace/item/3136...,https://scontent-sjc3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
...,...,...,...,...,...,...,...,...,...,...,...,...
115,2024-03-29,Honda Insight Bumper Protectir,0,Sunnyvale,CA,0,https://www.facebook.com/marketplace/item/3661...,https://scontent-sjc3-1.xx.fbcdn.net/v/t45.532...,True,False,False,fb
116,2024-03-29,2011 Honda civic Hybrid Sedan 4D,7300,Elk Grove,CA,98000,https://www.facebook.com/marketplace/item/1317...,https://scontent-sjc3-1.xx.fbcdn.net/v/t39.308...,False,False,False,fb
117,2024-03-29,2015 Honda civic EX Coupe 2D,6900,Oakland,CA,104000,https://www.facebook.com/marketplace/item/2022...,https://scontent-sjc3-1.xx.fbcdn.net/v/t39.308...,False,False,False,fb
118,2024-03-29,2011 Honda cr-z,5300,Suisun City,CA,212000,https://www.facebook.com/marketplace/item/7579...,https://scontent-sjc3-1.xx.fbcdn.net/v/t39.308...,False,False,False,fb


In [101]:
filtered_df = vehicles_df[vehicles_df['insight']==True]

In [102]:
filtered_df

,date,title,price,city,state,mileage,url,image,insight,first_gen,parts,site
0,2024-03-29,2010 Honda insight EX Hatchback 4D,4000,San Mateo,CA,164000,https://www.facebook.com/marketplace/item/3920...,https://scontent-sjc3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
1,2024-03-29,2020 Honda insight Touring Sedan 4D,15900,Benicia,CA,62000,https://www.facebook.com/marketplace/item/1543...,https://scontent-sjc3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
2,2024-03-29,2002 Honda insight Hybrid Hatchback 2D,3650,San Francisco,CA,165000,https://www.facebook.com/marketplace/item/1492...,https://scontent-sjc3-1.xx.fbcdn.net/v/t39.308...,True,True,False,fb
3,2024-03-29,2022 Honda insight EX Sedan 4D,24500,San Francisco,CA,24000,https://www.facebook.com/marketplace/item/4389...,https://scontent-sjc3-1.xx.fbcdn.net/v/t45.532...,True,False,False,fb
4,2024-03-29,2010 Honda insight EX Hatchback 4D,4500,Fremont,CA,183000,https://www.facebook.com/marketplace/item/3136...,https://scontent-sjc3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
...,...,...,...,...,...,...,...,...,...,...,...,...
108,2024-03-29,2020 Honda insight Sedan 4D,2000,Stockton,CA,70000,https://www.facebook.com/marketplace/item/1140...,https://scontent-sjc3-1.xx.fbcdn.net/v/t45.532...,True,False,False,fb
109,2024-03-29,2020 Honda insight EX Sedan 4D,22961,Hayward,CA,61000,https://www.facebook.com/marketplace/item/7435...,https://scontent-sjc3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
110,2024-03-29,2010 Honda Insight Engine,350,Concord,CA,0,https://www.facebook.com/marketplace/item/9355...,https://scontent-sjc3-1.xx.fbcdn.net/v/t45.532...,True,False,False,fb
111,2024-03-29,2021 Honda insight,23999,San Jose,CA,91000,https://www.facebook.com/marketplace/item/3576...,https://scontent-sjc3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb


In [103]:
csv_file_path = f'/Users/evanishibashi/Projects/insight/csv/fb/{date.today()}.csv'

filtered_df.to_csv(csv_file_path, index=False)